<font size="+3"> Conditional Time-series Generative Adversarial Network (cTimeGAN)</font>

# Imports & Settings

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import tensorflow as tf
from pathlib import Path
from tqdm import tqdm
import os

from keras.models import Sequential, Model
from keras.layers import GRU, Dense, RNN, GRUCell, Input, LSTM, Embedding, Reshape, Concatenate, CategoryEncoding
from keras.losses import BinaryCrossentropy, MeanSquaredError
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.utils import plot_model

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

In [ ]:
sns.set_style('white')

# Experiment Path

In [ ]:
results_path = Path('time_gan')
if not results_path.exists():
    results_path.mkdir()

In [ ]:
experiment = 0

In [ ]:
log_dir = results_path / f'experiment_{experiment:02}'
if not log_dir.exists():
    log_dir.mkdir(parents=True)

In [ ]:
hdf_store = results_path / 'TimeSeriesGAN.h5'

# Prepare Data

## Parameters

In [ ]:
seq_len = 200
n_seq = 13
batch_size = 128

feature_columns = ['Ipv', 'Vpv', 'Vdc', 'ia', 'ib', 'ic', 'va', 'vb', 'vc', 'Iabc', 'If', 'Vabc', 'Vf']

## Normalize Data

In [ ]:
dataset_folder = '/kaggle/input/gpvs-ts-npy'

x_train = np.load(os.path.join(dataset_folder, 'X_train.npy'))
y_train = np.load(os.path.join(dataset_folder, 'y_train.npy'))

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape).astype(np.float32)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(np.ravel(y_train))

n_classes = len(label_encoder.classes_)
print(x_train.shape, y_train.shape)

## Create tf.data.Dataset

In [ ]:
real_series = (tf.data.Dataset
               .from_tensor_slices((x_train, y_train))
               .shuffle(buffer_size=len(x_train))
               .batch(batch_size, drop_remainder=True))
real_series_iter = iter(real_series.repeat())

## Set up random series generator

In [ ]:
def make_random_data():
    while True:
        yield np.random.uniform(low=0, high=1, size=(seq_len, n_seq))

We use the Python generator to feed a `tf.data.Dataset` that continues to call the random number generator as long as necessary and produces the desired batch size.

In [ ]:
random_series = iter(tf.data.Dataset
                     .from_generator(make_random_data, output_types=tf.float32)
                     .batch(batch_size)
                     .repeat())

# TimeGAN Components

The design of the TimeGAN components follows the author's sample code.

##  Network Parameters

In [ ]:
hidden_dim = 4*n_seq
latent_dim = 7
num_layers = 3
lr = 1e-4

## Set up logger

In [ ]:
writer = tf.summary.create_file_writer(log_dir.as_posix())

## Input place holders

In [ ]:
X = Input(shape=[seq_len, n_seq], name='RealData')
Y = Input(shape=[1,], name='Label')
Z = Input(shape=[seq_len, n_seq], name='RandomData')

In [ ]:
def make_conditional_input(n_classes, seq_len, n_seq):
    return Sequential([Embedding(
                        input_dim=1,
                        output_dim=seq_len),
                       Reshape((seq_len, 1))], name = 'CondInput')

def make_category_encoding(n_classes, seq_len, n_seq):
    return Sequential([CategoryEncoding(num_tokens=n_classes, output_mode="one_hot"),
                      Embedding(
                        input_dim=n_classes,
                        output_dim=seq_len),
                       Reshape((seq_len, n_classes)),
                       Dense(n_classes//2)], name = 'CondInput')

In [ ]:
cond_input = make_conditional_input(n_classes, seq_len, n_seq)

## RNN block generator

We keep it very simple and use a very similar architecture for all four components. For a real-world application, they should be tailored to the data.

In [ ]:
def make_rnn(n_layers, hidden_units, output_units, name):
    return Sequential([GRU(units=hidden_units,
                           return_sequences=True,
                           name=f'GRU_{i + 1}') for i in range(n_layers)] +
                      [Dense(units=output_units,
                             activation='sigmoid',
                             name='OUT')], name=name)

## Embedder & Recovery

In [ ]:
embedder = make_rnn(n_layers=3,
                    hidden_units=hidden_dim,
                    output_units=latent_dim,
                    name='Embedder')
recovery = make_rnn(n_layers=3,
                    hidden_units=hidden_dim,
                    output_units=n_seq,
                    name='Recovery')

## Generator & Discriminator

In [ ]:
generator = make_rnn(n_layers=3,
                     hidden_units=hidden_dim,
                     output_units=latent_dim,
                     name='Generator')
discriminator = make_rnn(n_layers=3,
                         hidden_units=hidden_dim,
                         output_units=1,
                         name='Discriminator')
supervisor = make_rnn(n_layers=2,
                      hidden_units=hidden_dim,
                      output_units=latent_dim,
                      name='Supervisor')

# TimeGAN Training

## Settings

In [ ]:
train_steps = 20000
gamma = 1

## Generic Loss Functions

In [ ]:
mse = MeanSquaredError()
bce = BinaryCrossentropy()

# Phase 1: Autoencoder Training

## Architecture

In [ ]:
H = embedder(X)
X_tilde = recovery(H)

autoencoder = Model(inputs=X,
                    outputs=X_tilde,
                    name='Autoencoder')

In [ ]:
autoencoder.summary()

In [ ]:
plot_model(autoencoder,
           to_file=(results_path / 'autoencoder.png').as_posix(),
           show_shapes=True)

## Autoencoder Optimizer

In [ ]:
autoencoder_optimizer = Adam(learning_rate=lr)

## Autoencoder Training Step

In [ ]:
@tf.function
def train_autoencoder_init(x):
    with tf.GradientTape() as tape:
        x_tilde = autoencoder(x)
        embedding_loss_t0 = mse(x, x_tilde)
        e_loss_0 = 10 * tf.sqrt(embedding_loss_t0)

    var_list = embedder.trainable_variables + recovery.trainable_variables
    gradients = tape.gradient(e_loss_0, var_list)
    autoencoder_optimizer.apply_gradients(zip(gradients, var_list))
    return tf.sqrt(embedding_loss_t0)

## Autoencoder Training Loop

In [ ]:
for step in tqdm(range(train_steps)):
    X_, Y_ = next(real_series_iter)
    step_e_loss_t0 = train_autoencoder_init(X_)
    with writer.as_default():
        tf.summary.scalar('Loss Autoencoder Init', step_e_loss_t0, step=step)

## Persist model

In [ ]:
# autoencoder.save(log_dir / 'autoencoder')

# Phase 2: Supervised training

## Define Optimizer

In [ ]:
supervisor_optimizer = Adam(learning_rate=lr)

## Train Step

In [ ]:
@tf.function
def train_supervisor(x):
    with tf.GradientTape() as tape:
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        g_loss_s = mse(h[:, 1:, :], h_hat_supervised[:, :-1, :])

    var_list = supervisor.trainable_variables
    gradients = tape.gradient(g_loss_s, var_list)
    supervisor_optimizer.apply_gradients(zip(gradients, var_list))
    return g_loss_s

## Training Loop

In [ ]:
for step in tqdm(range(train_steps)):
    X_, Y_ = next(real_series_iter)
    step_g_loss_s = train_supervisor(X_)
    with writer.as_default():
        tf.summary.scalar('Loss Generator Supervised Init', step_g_loss_s, step=step)

## Persist Model

In [ ]:
# supervisor.save(log_dir / 'supervisor')

# Joint Training

## Generator

### Adversarial Architecture - Supervised

In [ ]:
Y_cond = cond_input(Y)
gen_input = Concatenate()([Z, Y_cond])

E_hat = generator(gen_input)
H_hat = supervisor(E_hat)

dis_input = Concatenate()([H_hat, Y_cond])
Y_fake = discriminator(dis_input)

adversarial_supervised = Model(inputs=[Z, Y],
                               outputs=Y_fake,
                               name='AdversarialNetSupervised')

In [ ]:
adversarial_supervised.summary()

In [ ]:
plot_model(adversarial_supervised, show_shapes=True)

### Adversarial Architecture in Latent Space

In [ ]:
dis_input_e = Concatenate()([E_hat, Y_cond])
Y_fake_e = discriminator(dis_input_e)

adversarial_emb = Model(inputs=[Z, Y],
                    outputs=Y_fake_e,
                    name='AdversarialNet')

In [ ]:
adversarial_emb.summary()

In [ ]:
plot_model(adversarial_emb, show_shapes=True)

### Mean & Variance Loss

In [ ]:
X_hat = recovery(H_hat)
synthetic_data = Model(inputs=[Z, Y],
                       outputs=X_hat,
                       name='SyntheticData')

In [ ]:
synthetic_data.summary()

In [ ]:
plot_model(synthetic_data, show_shapes=True)

In [ ]:
def get_generator_moment_loss(y_true, y_pred):
    y_true_mean, y_true_var = tf.nn.moments(x=y_true, axes=[0])
    y_pred_mean, y_pred_var = tf.nn.moments(x=y_pred, axes=[0])
    g_loss_mean = tf.reduce_mean(tf.abs(y_true_mean - y_pred_mean))
    g_loss_var = tf.reduce_mean(tf.abs(tf.sqrt(y_true_var + 1e-6) - tf.sqrt(y_pred_var + 1e-6)))
    return g_loss_mean + g_loss_var

## Discriminator

### Architecture: Real Data

In [ ]:
Y_real = discriminator(Concatenate()([H, Y_cond]))
discriminator_model = Model(inputs=[X,Y],
                            outputs=Y_real,
                            name='DiscriminatorReal')

In [ ]:
discriminator_model.summary()

In [ ]:
plot_model(discriminator_model, show_shapes=True)

## Optimizers

In [ ]:
generator_optimizer = Adam(learning_rate=lr)
discriminator_optimizer = Adam(learning_rate=lr)
embedding_optimizer = Adam(learning_rate=lr)

## Generator Train Step

In [ ]:
@tf.function
def train_generator(x, z, y):
    with tf.GradientTape() as tape:
        y_fake = adversarial_supervised([z, y])
        generator_loss_unsupervised = bce(y_true=tf.ones_like(y_fake),
                                          y_pred=y_fake)

        y_fake_e = adversarial_emb([z, y])
        generator_loss_unsupervised_e = bce(y_true=tf.ones_like(y_fake_e),
                                            y_pred=y_fake_e)
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        generator_loss_supervised = mse(h[:, 1:, :], h_hat_supervised[:, 1:, :])

        x_hat = synthetic_data([z, y])
        generator_moment_loss = get_generator_moment_loss(x, x_hat)

        generator_loss = (generator_loss_unsupervised +
                          generator_loss_unsupervised_e +
                          100 * tf.sqrt(generator_loss_supervised) +
                          100 * generator_moment_loss)

    var_list = generator.trainable_variables + supervisor.trainable_variables
    gradients = tape.gradient(generator_loss, var_list)
    generator_optimizer.apply_gradients(zip(gradients, var_list))
    return generator_loss_unsupervised, generator_loss_supervised, generator_moment_loss

## Embedding Train Step

In [ ]:
@tf.function
def train_embedder(x):
    with tf.GradientTape() as tape:
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        generator_loss_supervised = mse(h[:, 1:, :], h_hat_supervised[:, 1:, :])

        x_tilde = autoencoder(x)
        embedding_loss_t0 = mse(x, x_tilde)
        e_loss = 10 * tf.sqrt(embedding_loss_t0) + 0.1 * generator_loss_supervised

    var_list = embedder.trainable_variables + recovery.trainable_variables
    gradients = tape.gradient(e_loss, var_list)
    embedding_optimizer.apply_gradients(zip(gradients, var_list))
    return tf.sqrt(embedding_loss_t0)

## Discriminator Train Step

In [ ]:
@tf.function
def get_discriminator_loss(x, z, y):
    y_real = discriminator_model([x,y])
    discriminator_loss_real = bce(y_true=tf.ones_like(y_real),
                                  y_pred=y_real)

    y_fake = adversarial_supervised([z, y])
    discriminator_loss_fake = bce(y_true=tf.zeros_like(y_fake),
                                  y_pred=y_fake)

    y_fake_e = adversarial_emb([z, y])
    discriminator_loss_fake_e = bce(y_true=tf.zeros_like(y_fake_e),
                                    y_pred=y_fake_e)
    return (discriminator_loss_real +
            discriminator_loss_fake +
            gamma * discriminator_loss_fake_e)

In [ ]:
@tf.function
def train_discriminator(x, z, y):
    with tf.GradientTape() as tape:
        discriminator_loss = get_discriminator_loss(x, z, y)

    var_list = discriminator.trainable_variables
    gradients = tape.gradient(discriminator_loss, var_list)
    discriminator_optimizer.apply_gradients(zip(gradients, var_list))
    return discriminator_loss

## Training Loop

In [ ]:
step_g_loss_u = step_g_loss_s = step_g_loss_v = step_e_loss_t0 = step_d_loss = 0
for step in range(1,train_steps+1):
    # Train generator (twice as often as discriminator)
    for kk in range(2):
        X_, Y_ = next(real_series_iter)
        Z_ = next(random_series)

        # Train generator
        step_g_loss_u, step_g_loss_s, step_g_loss_v = train_generator(X_, Z_, Y_)
        # Train embedder
        step_e_loss_t0 = train_embedder(X_)

    X_, Y_ = next(real_series_iter)
    Z_ = next(random_series)
    step_d_loss = get_discriminator_loss(X_, Z_, Y_)
    if step_d_loss > 0.15:
        step_d_loss = train_discriminator(X_, Z_, Y_)

    if step % 1000 == 0:
        print(f'{step:6,.0f} | d_loss: {step_d_loss:6.4f} | g_loss_u: {step_g_loss_u:6.4f} | '
              f'g_loss_s: {step_g_loss_s:6.4f} | g_loss_v: {step_g_loss_v:6.4f} | e_loss_t0: {step_e_loss_t0:6.4f}')

    with writer.as_default():
        tf.summary.scalar('G Loss S', step_g_loss_s, step=step)
        tf.summary.scalar('G Loss U', step_g_loss_u, step=step)
        tf.summary.scalar('G Loss V', step_g_loss_v, step=step)
        tf.summary.scalar('E Loss T0', step_e_loss_t0, step=step)
        tf.summary.scalar('D Loss', step_d_loss, step=step)

## Persist Synthetic Data Generator

In [ ]:
synthetic_data.save(log_dir / 'synthetic_data')

# Generate Synthetic Data

In [ ]:
x_test = np.load(os.path.join(dataset_folder, 'X_test.npy'))
y_test = np.load(os.path.join(dataset_folder, 'y_test.npy'))

#x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape).astype(np.float32)
y_test = label_encoder.transform(np.ravel(y_test))

In [ ]:
test_data = (tf.data.Dataset
               .from_tensor_slices((x_test, y_test))
               .shuffle(buffer_size=len(x_test))
               .batch(batch_size, drop_remainder=True))

In [ ]:
generated_data = []
labels = []

for X_, Y_ in tqdm(test_data):
    Z_ = next(random_series)
    d = synthetic_data([Z_, Y_])
    generated_data.append(d)
    labels.append(Y_)

In [ ]:
generated_data = np.array(np.vstack(generated_data))
labels = np.array(np.hstack(labels))

generated_data.shape, labels.shape

## Rescale

In [ ]:
generated_data = (scaler.inverse_transform(generated_data
                  .reshape(-1, generated_data.shape[-1]))
                  .reshape(generated_data.shape))
generated_data.shape

In [ ]:
np.save(log_dir / 'generated_data.npy', generated_data)
np.save(log_dir / 'generated_labels.npy', labels)

## Persist Data

In [ ]:
with pd.HDFStore(hdf_store) as store:
    store.put('data/synthetic', pd.DataFrame(generated_data.reshape(-1, n_seq),
                                             columns=feature_columns))

## Plot sample Series

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(14, 7))
axes = axes.flatten()

idx = np.random.randint(generated_data.shape[0])
synthetic = generated_data[idx]
label = labels[idx]

x_test_label = x_test[y_test == label]
real = x_test_label[np.random.randint(x_test_label.shape[0]), :, :]

for j, ticker in enumerate(feature_columns):
    (pd.DataFrame({'Real': real[:, j],
                   'Synthetic': synthetic[:, j]})
     .plot(ax=axes[j],
           title=ticker,
           secondary_y='Synthetic', style=['-', '--'],
           lw=1))
sns.despine()
fig.suptitle(f'Label: {label}')
fig.tight_layout()

# Evaluation

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

def visualization(ori_data, generated_data, analysis):
  """Using PCA or tSNE for generated and original data visualization.
  
  Args:
    - ori_data: original data
    - generated_data: generated synthetic data
    - analysis: tsne or pca
  """  
  # Analysis sample size (for faster computation)
  lenght = min([len(ori_data), len(generated_data)])
  anal_sample_no = min([1000, lenght])
  idx = np.random.permutation(lenght)[:anal_sample_no]
    
  # Data preprocessing
  ori_data = np.asarray(ori_data)
  generated_data = np.asarray(generated_data)  
  
  ori_data = ori_data[idx]
  generated_data = generated_data[idx]
  
  no, seq_len, dim = ori_data.shape  
  
  for i in range(anal_sample_no):
    if (i == 0):
      prep_data = np.reshape(np.mean(ori_data[0,:,:], 1), [1,seq_len])
      prep_data_hat = np.reshape(np.mean(generated_data[0,:,:],1), [1,seq_len])
    else:
      prep_data = np.concatenate((prep_data, 
                                  np.reshape(np.mean(ori_data[i,:,:],1), [1,seq_len])))
      prep_data_hat = np.concatenate((prep_data_hat, 
                                      np.reshape(np.mean(generated_data[i,:,:],1), [1,seq_len])))
    
  # Visualization parameter        
  colors = ["red" for i in range(anal_sample_no)] + ["blue" for i in range(anal_sample_no)]    
    
  if analysis == 'pca':
    # PCA Analysis
    pca = PCA(n_components = 2)
    pca.fit(prep_data)
    pca_results = pca.transform(prep_data)
    pca_hat_results = pca.transform(prep_data_hat)
    
    # Plotting
    f, ax = plt.subplots(1)    
    plt.scatter(pca_results[:,0], pca_results[:,1],
                c = colors[:anal_sample_no], alpha = 0.2, label = "Original")
    plt.scatter(pca_hat_results[:,0], pca_hat_results[:,1], 
                c = colors[anal_sample_no:], alpha = 0.2, label = "Synthetic")
  
    ax.legend()  
    plt.title('PCA plot')
    plt.xlabel('x-pca')
    plt.ylabel('y_pca')
    plt.show()
    
  elif analysis == 'tsne':
    
    # Do t-SNE Analysis together       
    prep_data_final = np.concatenate((prep_data, prep_data_hat), axis = 0)
    
    # TSNE anlaysis
    tsne = TSNE(n_components = 2, verbose = 1, perplexity = 40, n_iter = 300)
    tsne_results = tsne.fit_transform(prep_data_final)
      
    # Plotting
    f, ax = plt.subplots(1)
      
    plt.scatter(tsne_results[:anal_sample_no,0], tsne_results[:anal_sample_no,1], 
                c = colors[:anal_sample_no], alpha = 0.2, label = "Original")
    plt.scatter(tsne_results[anal_sample_no:,0], tsne_results[anal_sample_no:,1], 
                c = colors[anal_sample_no:], alpha = 0.2, label = "Synthetic")
  
    ax.legend()
      
    plt.title('t-SNE plot')
    plt.xlabel('x-tsne')
    plt.ylabel('y_tsne')
    plt.show()

In [ ]:
visualization(x_test, generated_data, 'pca')

In [ ]:
visualization(x_test, generated_data, 'tsne')